In [26]:
!pip install pymupdf pytesseract sentence-transformers faiss-cpu transformers pillow -q
!apt-get update && apt-get install -y tesseract-ocr tesseract-ocr-ben -q

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists...
Building dependency tree...
Reading state inf

In [27]:
!pip install pyngrok

In [28]:
import re
import unicodedata
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import fitz  # PyMuPDF
from PIL import Image
import pytesseract
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from fastapi import FastAPI, File, UploadFile
import nest_asyncio
from pyngrok import ngrok, conf
import uvicorn
import logging
import os
from pathlib import Path

In [29]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [30]:
# Initialize models
try:
    embedding_model = SentenceTransformer('l3cube-pune/bengali-sentence-similarity-sbert')
    tokenizer_hf = AutoTokenizer.from_pretrained('csebuetnlp/banglat5')
    model_hf = AutoModelForSeq2SeqLM.from_pretrained('csebuetnlp/banglat5')
except Exception as e:
    logger.error(f"Failed to initialize models: {e}")
    raise

In [31]:
# Global variables
chunks = []
index = None
embeddings = None
conversation_history = []

In [32]:
# ## Functions
def preprocess_image(img):
    """Preprocess image for better OCR results."""
    try:
        # Convert to grayscale
        img = img.convert('L')
        # Enhance contrast
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(2.5)
        # Sharpen image
        img = img.filter(ImageFilter.SHARPEN)
        # Resize for higher resolution
        img = img.resize((int(img.width * 2.5), int(img.height * 2.5)), Image.Resampling.LANCZOS)
        return img
    except Exception as e:
        logger.error(f"Error preprocessing image: {e}")
        return img

In [33]:
def clean_text(text):
    """Clean extracted text to remove noise and normalize Bangla characters."""
    try:
        # Normalize Unicode
        text = unicodedata.normalize('NFKC', text)
        # Keep only Bangla characters, punctuation, and whitespace
        text = re.sub(r'[^\u0980-\u09FF।!?\s\n\d]', '', text)
        # Remove stray vowel signs and invalid sequences
        text = re.sub(r'[\u09BE-\u09CC\u09D7]+(?![া-ঢ়য়-৺])', '', text)
        # Normalize spaces and newlines
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\n+', '\n', text).strip()
        # Filter short lines (likely noise)
        lines = [line for line in text.split('\n') if len(line.strip()) > 10]
        return '\n'.join(lines)
    except Exception as e:
        logger.error(f"Error cleaning text: {e}")
        return text

In [34]:
def extract_text_from_pdf(pdf_path):
    """Extract text from PDF with enhanced OCR and cleaning."""
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            page_text = page.get_text()
            if len(page_text.strip()) < 50:  # Likely a scanned page
                pix = page.get_pixmap(dpi=400)
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.rgb)
                img = preprocess_image(img)
                page_text = pytesseract.image_to_string(img, lang='ben', config='--psm 6 --oem 3')
            text += clean_text(page_text) + "\n"
        doc.close()
        # Save extracted text for debugging
        with open('extracted_text.txt', 'w', encoding='utf-8') as f:
            f.write(text)
        # Check text quality
        bangla_chars = len(re.findall(r'[\u0980-\u09FF]', text))
        total_chars = len(text.replace('\n', ''))
        if total_chars > 0 and bangla_chars / total_chars < 0.6:
            logger.warning("Extracted text may be noisy; Bangla character ratio low: {:.2f}".format(bangla_chars / total_chars))
        return text
    except Exception as e:
        logger.error(f"Error extracting text from PDF: {e}")
        return ""

In [35]:
def chunk_text(text, max_tokens=200):
    """Chunk text into question-answer pairs or sentences, handling noisy text."""
    chunks = []
    question_pattern = r'(\d+\.\s*.*?[।!?])\s*(ক\s*.*?|খ\s*.*?|গ\s*.*?|ঘ\s*.*?(?=\d+\.|$))'
    matches = re.finditer(question_pattern, text, re.DOTALL | re.IGNORECASE)

    for match in matches:
        qa_pair = match.group(0).strip()
        token_count = len(qa_pair.split())
        if token_count <= max_tokens:
            chunks.append(qa_pair)
        else:
            sentences = re.split(r'(?<=[।!?\.])\s+', qa_pair)
            sub_chunk = ""
            sub_tokens = 0
            for sentence in sentences:
                sentence = sentence.strip()
                if not sentence or len(sentence) < 10:
                    continue
                sentence_tokens = len(sentence.split())
                if sub_tokens + sentence_tokens <= max_tokens:
                    sub_chunk += " " + sentence
                    sub_tokens += sentence_tokens
                else:
                    if sub_chunk:
                        chunks.append(sub_chunk.strip())
                    sub_chunk = sentence
                    sub_tokens = sentence_tokens
            if sub_chunk:
                chunks.append(sub_chunk.strip())

    remaining_text = re.sub(question_pattern, '', text, flags=re.DOTALL | re.IGNORECASE)
    sentences = re.split(r'(?<=[।!?\.])\s+', remaining_text)
    current_chunk = ""
    current_tokens = 0
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence or len(sentence) < 10:
            continue
        sentence_tokens = len(sentence.split())
        if current_tokens + sentence_tokens <= max_tokens:
            current_chunk += " " + sentence
            current_tokens += sentence_tokens
        else:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = sentence
            current_tokens = sentence_tokens
    if current_chunk:
        chunks.append(current_chunk.strip())

    return [chunk for chunk in chunks if chunk and len(chunk.strip()) > 15]

In [36]:
def preprocess_query(query):
    """Normalize and clean the query text."""
    query = unicodedata.normalize('NFKC', query)
    query = re.sub(r'\s+', ' ', query).strip()
    return query

In [37]:
def generate_answer_from_context(context, question):
    """Generate an answer with strict option selection."""
    try:
        prompt = (
            f"প্রশ্ন: {question}\n"
            f"প্রসঙ্গ: {context}\n"
            f"উত্তর: শুধুমাত্র সঠিক বিকল্পটি নির্বাচন করুন (যেমন ক, খ, গ, ঘ) এবং বিকল্পের পাঠ্য। "
            f"অন্য কোনো পাঠ্য যোগ করবেন না।"
        )
        inputs = tokenizer_hf(prompt, return_tensors="pt", truncation=True, max_length=512)
        outputs = model_hf.generate(
            **inputs,
            max_new_tokens=50,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        answer = tokenizer_hf.decode(outputs[0], skip_special_tokens=True)
        return answer.strip()
    except Exception as e:
        logger.error(f"Error generating answer: {e}")
        return "Error generating answer."

In [38]:
def process_pdf(pdf_path):
    """Process a PDF to update global chunks, index, and embeddings."""
    global chunks, index, embeddings
    text = extract_text_from_pdf(pdf_path)
    if not text:
        logger.error("No text extracted from PDF.")
        return False

    chunks = chunk_text(text, max_tokens=200)
    if not chunks:
        logger.error("No chunks created from text.")
        return False

    embeddings = embedding_model.encode(chunks)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings.astype("float32"))
    logger.info(f"Processed PDF: {pdf_path}, created {len(chunks)} chunks")
    return True

In [39]:
def retrieve_and_answer(query, chunks, index, embeddings, top_k=10, threshold=0.3):
    """Retrieve relevant chunks and generate an answer."""
    global conversation_history
    try:
        if not chunks or index is None or embeddings is None:
            logger.warning("No PDF data available.")
            return "No PDF data available.", ""

        query = preprocess_query(query)
        query_vec = embedding_model.encode([query]).astype("float32")
        distances, indices = index.search(query_vec, top_k)

        if min(distances[0]) > threshold:
            logger.warning("No relevant context found.")
            return "Query is too vague or no relevant information found.", ""

        context = "\n".join([chunks[i] for i in indices[0] if re.search(r'[\u0980-\u09FF]', chunks[i])])
        if not context:
            logger.warning("No valid Bangla context found.")
            return "No valid context found.", ""

        answer = generate_answer_from_context(context, query)
        conversation_history.append({
            "query": query,
            "answer": answer,
            "context": context[:300],
            "grounded": evaluate_groundedness(answer, context)
        })
        return answer, context
    except Exception as e:
        logger.error(f"Error in retrieve_and_answer: {e}")
        return "Error processing query.", str(e)

In [40]:
# ## Upload PDF
print("Please upload HSC26-Bangla1st-Paper.pdf")
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# ## Specific Test Cases
# Define test cases
test_cases = [
    {"query": "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?", "expected_answer": "শুম্ভুনাথ"},
    {"query": "কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?", "expected_answer": "মামাকে"},
    {"query": "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?", "expected_answer": "১৫ বছর"}
]

Please upload HSC26-Bangla1st-Paper.pdf


Saving HSC26-Bangla1st-Paper.pdf to HSC26-Bangla1st-Paper.pdf


In [45]:
# Process the PDF
success = process_pdf("/content/HSC26-Bangla1st-Paper.pdf")
if not success:
    print("Failed to process PDF. Check logs and 'extracted_text.txt' for details.")
else:
    print(f"PDF processed successfully: {len(chunks)} chunks created.")

PDF processed successfully: 35 chunks created.


In [46]:
# Run tests
for test in test_cases:
    query = test["query"]
    expected_answer = test["expected_answer"]

    print("\n" + "="*60)
    print(f"Query: {query}")
    answer, context = retrieve_and_answer(query, chunks, index, embeddings, top_k=10, threshold=0.3)
    grounded = evaluate_groundedness(answer, context)

    # Check if the answer matches the expected answer
    pass_match = expected_answer in answer
    status = "Passed" if pass_match else "Failed"

    print(f"Generated Answer: {answer}")
    print(f"Expected Answer: {expected_answer}")
    print(f"Context (truncated): {context[:300]}...")
    print(f"Grounded: {grounded}")
    print(f"Status: {status}")
    print("="*60)


Query: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?
Generated Answer: পপব্সকহব? ক খ গ ণযববববব গ কব ক গ গ ৩৮। ক কখখগ কগ গ ম গ ন গবগ
Expected Answer: শুম্ভুনাথ
Context (truncated): ক অনপকম মম খ অনপকম ম গ ম্ভনথ ব্ব্ ঘ হর্ উ গ ৩৩। র্র্কনসকথকলযণর্মলযকথ ? উভ ইর্জক্ষত উভ ইর্জক্ষত ব্ব্আজ্ঞব্হ র্নকচ যকনট সঠক? ক খ গ ঘ উ ক র্নকচউদ্দপকটপক়ে৩৪ও৩৫নম্বপ্রকউদও স্বতসর্জক্ষতওআত্মর্নভলন।র্ব্ক পশ্বশওশর়্চকপচকর়্কতব্ধযহ ।শ্বশ শর়্ধণচকর্ব্ব্উঅহংকহ ।তসংসকপ্রর্তদর্ ত্বলন ।য দ ১৯ ৩৪। অপর্র্চত গকসকঙ্...
Grounded: False
Status: Failed

Query: কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?
Generated Answer: ণ কণক কগ ক গ খ খ গ গ ঘ ৪৩ গ ৪৯ ক খ ৪২ গ ৫৩ গ ৪৬ খ ৪৯ গ ৪৭ ক ৪২ ক ৪৩ ক ৪১ ক ৪৯ খ ক্্্ কক্ ক্
Expected Answer: মামাকে
Context (truncated): ক খ গ ঘ
উমল ১ ক ২ খ ৩ ক ৪ গ ৫ ক ৬ গ ৭ ক ৮ খ ৯ খ ১০ খ ১১ খ ১২ ক ১৩ গ ১৪ খ ১৫ গ ১৬ গ ১৭ খ ১৮ খ ১৯ গ ২০ ঘ ২১ ঘ ২২ ক ২৩ ঘ ২৪ ক ২৫ ঘ ২৬ খ ২৭ ক ২৮ ক ২৯ গ ৩০ ক ৩১ গ ৩২ খ ৩৩ খ ৩৪ ক ৩৫ ঘ ৩৬ খ ৩৭ ক ৩৮ গ ৩৯ গ ৪০ খ ৪১ গ ৪২ ক ৪৩ ক ৪৪ গ ৪৫ খ ৪৬ খ ৪৭ খ ৪৮ খ ৪৯ খ ৫০ খ ৫১ ঘ ৫২ গ ৫৩ গ ৫৪ খ ৫৫ খ ৫

In [47]:
# ## Debugging Output
# Display first 1000 characters of extracted text for inspection
with open('extracted_text.txt', 'r', encoding='utf-8') as f:
    extracted_text = f.read()
print("\nExtracted Text (first 1000 characters):\n")
print(extracted_text[:1000])


Extracted Text (first 1000 characters):

অনলইন ব্যচ সম্পর্কত যকককন জজ্ঞস অপররত আল য রষয় ং ১ম পত্র
১। অনপল ক কল জরক রনবহ কলতন? ক ডক্তর্ খ ওকলর্ত গ মস্টর্ ঘ ব্যব্স ২। লক ভগ্য দত প্রধন এলজন্ট কণ ত ক প্রর্তপজ খ প্রভব্ গ র্ব্চক্ষণত ঘ কট ব্র্ র্নকচ অনকদট পক় ৩ ও ৪ সংখযক প্রক উ দও। র্পতহন দপ চচই র্কলন পর্ব্ক কত। দপ র্জক্ষত হকলও ত র্সন্ত যনও ক্ষমত র্ল ন। চচ ত র্ব্ক উকদযগ র্নকলও যতক র্নক ব়্ব্র়্ ক ককণ কনয র্পত অপমর্নত যব্ধ কক র্ব্ক আকলচন যভক যদন। দপ যমক ট র্ব্ যদকখ মগ্ধ হকলও ত চচকক র্কই ব্লকত পকনর্ন। ৩। প সল অপররত গ্ল দকন রল র আল? ক হর্ক খ মম গ র্ক্ষকক ঘ র্ব্ন ৪। উক্ত রল প্রধনয দপলয়ল যদত্ম হনম্মনযত যলভ র্নকচ যকনট ঠক? ক। ও খ। ও গ। ও ঘ। ও ৫ অনপল য়স কত ? ক পঁর্চ খ ব্ব গ সত ঘ আট প্রকমলয ন কতগকল প্রক সঠক উ র্দকত পকল? ১ খ ২ গ ৩ খ ৪ ক ৫ গ র্নম্নর্ব্ব্যজক্তহঠাৎর্ব্লহক ওঠফকলসমকপর্চ সংকটসম্পককধণলভককব্। তৎকলনসমসভযতওমনব্তঅব্মননসম্পককনকতপকব্। তৎকলনসমকপণপ্রথকপ্রভব্সম্পককনকতপকব্। তৎককলসমকভদ্রকলককস্বভব্বব্র্ষ্ট্যসম্পককজ্ঞনলভককব্। নযকমলঠক র্কন্তদব্লন কলযণব্নচর্তদ্বপ্রর্তজতএইসতযঅনধব্নককত পকব্। মনষআর্নক যব্ঁ

In [48]:
# Set up FastAPI
app = FastAPI()

@app.post("/upload-pdf")
async def upload_pdf(file: UploadFile = File(...)):
    """Upload a PDF file and process it for RAG."""
    try:
        if not file.filename.endswith('.pdf'):
            logger.error("Uploaded file is not a PDF.")
            return {"error": "Please upload a valid PDF file."}

        # Save the uploaded file temporarily
        temp_file = f"temp_{file.filename}"
        with open(temp_file, "wb") as f:
            content = await file.read()
            f.write(content)

        # Process the PDF
        success = process_pdf(temp_file)

        # Clean up the temporary file
        try:
            os.remove(temp_file)
        except Exception as e:
            logger.warning(f"Failed to delete temporary file {temp_file}: {e}")

        if success:
            return {"message": f"PDF {file.filename} processed successfully. {len(chunks)} chunks created."}
        else:
            return {"error": "Failed to process PDF."}
    except Exception as e:
        logger.error(f"Error uploading PDF: {e}")
        return {"error": "Failed to upload and process PDF."}

In [49]:
@app.post("/ask")
async def ask_question(query: str):
    """API endpoint to process queries."""
    try:
        answer, context = retrieve_and_answer(query, chunks, index, embeddings)
        return {
            "query": query,
            "answer": answer,
            "context": context[:300],
            "grounded": evaluate_groundedness
        }
    except Exception as e:
        logger.error(f"API error: {e}")
        return {"error": "Failed to process query."}

In [ ]:
# Run the API
if __name__ == "__main__":
    nest_asyncio.apply()
    try:
        ngrok.set_auth_token("30PAzjEoyE4OhrNY5oEis32EmpL_XWLYjfSCFn8K4TaocFLD")
        max_retries = 3
        for attempt in range(max_retries):
            try:
                public_url = ngrok.connect(8001)
                logger.info(f"Public URL: {ngrok.connect(8000)}")
                break
            except Exception as e:
                logger.warning(f"Ngrok connection attempt {attempt + 1} failed: {e}")
                if attempt < max_retries - 1:
                    time.sleep(2)
                else:
                    logger.error("Failed to establish ngrok tunnel after retries.")
                    raise

        # Start FastAPI server
        print(f"Public URL: {public_url}")
        logger.info("Starting FastAPI server on http://localhost:8001")
        uvicorn.run(app, host="localhost", port=8001)
    except Exception as e:
        logger.error(f"Failed to start API: {e}")
        raise

Public URL: NgrokTunnel: "https://7cd4560022b9.ngrok-free.app" -> "http://localhost:8001"


INFO:     Started server process [932]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8001 (Press CTRL+C to quit)
